# 3. Test Semantic Search with Semantic Text
- Link Doc tham khảo: https://www.elastic.co/guide/en/elasticsearch/reference/current/semantic-search.html

## 3.1. Thư viện

In [1]:
from elasticsearch import Elasticsearch, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

## 3.2 Khởi tạo Elasticsearch Client

In [2]:
# Trang chủ ElasticCloud: https://cloud.elastic.co/login?redirectTo=%2Fhome

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = "law_chatbot:dXMtZWFzdDQuZ2NwLmVsYXN0aWMtY2xvdWQuY29tOjQ0MyQ0MWJmNDlmMjUzMjQ0OTczODIzNTZlMzQxOTVhMzM4MCRkODk4OTNkMWE5MjI0MTdmOWZlYWJlNDdmNjEyZTdhZQ=="

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = "RC1jb2daRUJ1clhCLU5mb2YxbWs6VG84ZUk3STJTcVduNUd4dktuRU5mQQ=="

In [3]:
# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    timeout=None
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17448\96837577.py:2: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  client = Elasticsearch(


## 3.3. Create the Inference Endpoint

**Link Document:**
https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html

Elasticsearch exposes REST APIs that are used by the UI components and can be called directly to configure and access Elasticsearch features, including Inference APIs.

An inference endpoint enables you to use the corresponding machine learning model without manual deployment and apply it to your data at ingestion time through semantic text.

![Alt text](data\images\inference-landscape.png)

In [4]:
# Link Document: https://www.elastic.co/guide/en/elasticsearch/reference/current/delete-inference-api.html

try:
    client.inference.delete(inference_id="my-elser-endpoint")
except exceptions.NotFoundError:
    # Inference endpoint does not exist
    print("Inference endpoint does not exist")
    pass

In [5]:
# Link Docs:
# - Tạo Inference: https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html
# - Tạo Inference với ELSER – a retrieval model trained by Elastic: https://www.elastic.co/guide/en/elasticsearch/reference/current/infer-service-elser.html

try:
    client.options(
        request_timeout=60, max_retries=3, retry_on_timeout=True
    ).inference.put(
        task_type="sparse_embedding",
        inference_id="my-elser-endpoint",
        body={
            "service": "elser",
            "service_settings": {
                "num_allocations": 1, 
                "num_threads": 1
            },
        },
    )
    print("Inference endpoint created successfully")
except exceptions.BadRequestError as e:
    if e.error == "resource_already_exists_exception":
        print("Inference endpoint created successfully")
    else:
        raise e

Inference endpoint created successfully


In [6]:
# Link Docs: https://www.elastic.co/guide/en/elasticsearch/reference/current/get-inference-api.html

inference_endpoint_info = client.inference.get(
    inference_id="my-elser-endpoint",
)

print(inference_endpoint_info)

{'endpoints': [{'inference_id': 'my-elser-endpoint', 'task_type': 'sparse_embedding', 'service': 'elser', 'service_settings': {'num_allocations': 1, 'num_threads': 1, 'model_id': '.elser_model_2_linux-x86_64'}, 'task_settings': {}}]}


In [7]:
model_id = inference_endpoint_info["endpoints"][0]["service_settings"]["model_id"]

In [8]:
while True:
    status = client.ml.get_trained_models_stats(
        model_id=model_id,
    )

    deployment_stats = status["trained_model_stats"][0].get("deployment_stats")
    if deployment_stats is None:
        print("ELSER Model is currently being deployed.")
        time.sleep(5)
        continue

    nodes = deployment_stats.get("nodes")
    if nodes is not None and len(nodes) > 0:
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

ELSER Model has been successfully deployed.


## 3.4. Create the Index

ElasticSearch stores its data in one or more indices. Using analogies from the SQL
world, index is something similar to a database. It is used to store the documents
and read them from it. As we already mentioned, under the hood, ElasticSearch
uses Apache Lucene library to write and read the data from the index. What one
should remember about is that a single ElasticSearch index may be built of more
than a single Apache Lucene index, by using shards and replicas. (Source: Page 15, Mastering ElasticSearch_ Extend your knowledge on ElasticSearch, and querying and data handling, along with its internal working (2013, Packt Publishing), https://drive.google.com/file/d/1uU4-Qd_D9B3wzAsEbrKetqoQZCqNGtvy/view?usp=drive_link)

In [9]:
# Link Doc: https://www.elastic.co/guide/en/elasticsearch/reference/current/indices.html

client.indices.delete(index="semantic-text-qa-gd_dt_yt_100", ignore_unavailable=True)
client.indices.create(
    index="semantic-text-qa-gd_dt_yt_100",
    mappings={
        "properties": {
            "situation": {"type": "text", "copy_to": "situation_semantic"},
            "answer": {"type": "text"},
            "situation_semantic": {
                "type": "semantic_text",
                "inference_id": "my-elser-endpoint",
            },
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'semantic-text-qa-gd_dt_yt_100'})

## 3.5. Populate the Index

In [10]:
import json

# Đọc dữ liệu từ file .json
with open(r'data\tinh_huong\output.json', 'r', encoding='utf-8') as file:
    qa_gd_dt_yt_100 = json.load(file)

In [11]:
operations = []
for qa in qa_gd_dt_yt_100:
    operations.append({"index": {"_index": "semantic-text-qa-gd_dt_yt_100"}})
    operations.append(qa)
client.bulk(index="semantic-text-qa-gd_dt_yt_100", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 1720857309, 'items': [{'index': {'_index': 'semantic-text-qa-gd_dt_yt_100', '_id': 'Pef2gZEBurXB-Nfo-1pN', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'semantic-text-qa-gd_dt_yt_100', '_id': 'Puf2gZEBurXB-Nfo-1pN', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'semantic-text-qa-gd_dt_yt_100', '_id': 'P-f2gZEBurXB-Nfo-1pN', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'semantic-text-qa-gd_dt_yt_100', '_id': 'QOf2gZEBurXB-Nfo-1pN', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful'

## 3.5. Semantic Search

### 3.5.1. Pretty printing Elasticsearch search results

In [12]:
def pretty_search_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            score = hit["_score"]
            situation = hit["_source"]["situation"]
            answer = hit["_source"]["answer"]

            pretty_output = f"\nID: {id}\nScore: {score}\nSituation: {situation}\nAnswer: {answer}"

            print(pretty_output)

### 3.5.2. Semantic Search with the semantic Query

In [13]:
response = client.search(
    index="semantic-text-qa-gd_dt_yt_100",
    query={
        "semantic": {
            "field": "situation_semantic", 
            "query": "tôi vừa thi đại học xong, điểm khá cao và đang đăng ký xét nguyện vọng, thì có phải thực hiện đi nghĩa vụ không nếu địa phương yêu cầu?"
        }
    },
)

pretty_search_response(response)


ID: c-f2gZEBurXB-Nfo-1pN
Score: 34.47626
Situation: Tôi xin  hỏi, trẻ em mẫu giáo và học sinh theo học tại các trường đóng trên địa bàn xã, thị trấn thuộc xã khu vực I, II; có hộ khẩu ở thôn, bản đặc biệt khó khăn, xã khu vực III, nhưng bố mẹ có nhà cửa và đang sinh sống làm việc tại khu vực I thì có được hỗ trợ chi phí học tập theo Nghị định số 81/2021/NĐ-CP không?
Answer: Bộ Giáo dục và Đào tạo trả lời vấn đề này như sau:Khoản 4 Điều 18 Nghị định số81/2021/NĐ-CPquy định đối tượng được hưởng hỗ trợ chi phí học tập gồm: Trẻ em học mẫu giáo và học sinh phổ thông, học viên học tại cơ sở giáo dục thường xuyên theo chương trình giáo dục phổ thông ở thôn/bản đặc biệt khó khăn, xã khu vực III vùng dân tộc và miền núi, xã đặc biệt khó khăn vùng bãi ngang ven biển hải đảo theo quy định của cơ quan có thẩm quyền.Trường hợp trẻ em học mẫu giáo và học sinh đang theo học tại các trường đóng trên địa bàn xã, thị trấn thuộc xã khu vực I, II; có hộ khẩu thường trú ở thôn, bản đặc biệt khó khăn, xã k

**Nhận Xét:**
- Phương pháp Semantic Search này đôi khi sẽ cho ra kết quả không được quá tốt, chẳng hạn như ở trường hợp trên, kết quả mong muốn lại xếp ở vị trí số 2.
- Kết quả này cũng cho thấy điểm yếu của Semantic Search. Vì Semantic Search dựa trên sự tương đồng của vector, nên có đôi lúc câu hỏi truy vấn của người dùng có độ tương đồng với dữ liệu khác so với kết quả mong muốn (độ sai lệch có thể khá nhỏ). Đó là lý do tại sao ở ví dụ trên, kết quả mong muốn lại đứng ở vị trí số 2.

In [14]:
response = client.search(
    index="semantic-text-qa-gd_dt_yt_100",
    query={
        "bool": {
            "must": {
                "multi_match": {
                    "fields": ["situation", "answer"],
                    "query": "tôi vừa thi đại học xong, điểm khá cao và đang đăng ký xét nguyện vọng, thì có phải thực hiện đi nghĩa vụ không nếu địa phương yêu cầu?",
                    "boost": 1.5
                }
            },
            "should": {
                "semantic": {
                    "field": "situation_semantic",
                    "query": "tôi vừa thi đại học xong, điểm khá cao và đang đăng ký xét nguyện vọng, thì có phải thực hiện đi nghĩa vụ không nếu địa phương yêu cầu?",
                    "boost": 3.0
                }
            },
        }
    },
)

pretty_search_response(response)


ID: Pef2gZEBurXB-Nfo-1pN
Score: 134.07864
Situation: Tôi xin hỏi, học sinh vừa tốt nghiệp THPT khi xét tuyển vào các trường cao đẳng thì có được tạm miễn thực hiện nghĩa vụ quân sự không?
Answer: Bộ Quốc phòng trả lời vấn đề này như sau:Theo quy định tại Điểm g Khoản 1 Điều 41Luật Nghĩa vụ quân sựnăm 2015, những công dân sau đây được tạm hoãn gọi nhập ngũ:"g) Đang học tại cơ sở giáo dục phổ thông; đang được đào tạo trình độ đại học hệ chính quy thuộc cơ sở giáo dục đại học, trình độ cao đẳng hệ chính quy thuộc các cơ sở giáo dục nghề nghiệp trong một khóa đào tạo của một trình độ đào tạo".Như vậy, công dân thuộc trường hợp nêu tại Điểm g Khoản 1 Điều 41 Luật Nghĩa vụ quân sự năm 2015 được tạm hoãn từ ngày công dân đã làm xong thủ tục nhập học và đang học tập tại trường; chỉ được tạm hoãn gọi nhập ngũ trong một khóa đào tạo tập trung đầu tiên, còn các khóa học tiếp theo thì không được tạm hoãn.

ID: h-f2gZEBurXB-Nfo-1pN
Score: 129.90456
Situation: Tôi tốt nghiệp Cao đẳng sư phạm Toán -

**Nhận xét:**
- Những kết quả này chứng minh rằng việc áp dụng kỹ thuật tìm kiếm từ khóa có thể giúp tập trung kết quả, đồng thời vẫn giữ được nhiều lợi thế của tìm kiếm ngữ nghĩa. Trong ví dụ này, top 1 kết quả truy vấn đã trả về kết quả mà ta mong muốn.